In [1]:
#Imports

import os
import sys
import numpy as np

import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake

from pycharmm.lib import charmm as libcharmm

In [2]:
#Chains
def get_assigned_segid(file=None):
    # read the pdb to get the assigned segid for each chain
    fpdb = open(file,'r')
    for l in fpdb:
        if l.split()[0] == 'ATOM':
            segid=l.strip().split()[-1]
            break
    fpdb.close()
    return segid

pdbid = '/home/trowray/acrb/pdb/1iwg'

os.system('convpdb.pl -info {}.pdb > info'.format(pdbid))
os.system('grep SSBOND {}.pdb >> info'.format(pdbid))
fr = open('info', 'r')
chains = []
segids = []
disu = {}
for l in fr:
    if 'chains total' in l:
        nchains = l.split()[0]
    if l.split()[0]=='==':
        chains.append(l.split()[2])
    if l.split()[0]=='SSBOND':
        num = l.split()[1]
        res1 = l.split()[2]
        ch1 = l.split()[3]
        rn1 = l.split()[4]
        res2 = l.split()[5]
        ch2 = l.split()[6]
        rn2 = l.split()[7]
        disu[num] ={'residues':[res1,res2],
                    'chains':[ch1,ch2],
                    'resnums':[rn1,rn2]}
fr.close()

!cat info
os.system('rm info') # clean-up


3 chains total
== chain A
1-1006
xrange 28.144000 123.648000 : 95.504000
yrange 32.155000 95.708000 : 63.553000
zrange 256.347000 383.868000 : 127.521000
== chain B
1-1006
xrange 15.168000 101.410000 : 86.242000
yrange 2.687000 79.108000 : 76.421000
zrange 256.347000 383.868000 : 127.521000
== chain C
1-1006
xrange 50.194000 117.362000 : 67.168000
yrange -11.697000 82.650000 : 94.347000
zrange 256.347000 383.868000 : 127.521000


--------------------------------------------------------------------------
A process has executed an operation involving a call to the
"fork()" system call to create a child process.  Open MPI is currently
operating in a condition that could result in memory corruption or
other system errors; your job may hang, crash, or produce silent
data corruption.  The use of fork() (or system() or other calls that
create child processes) is strongly discouraged.

The process that invoked fork was:

  Local host:          [[42027,1],0] (PID 17801)

If you are *absolutely sure* that your application will successfully
and correctly survive a call to fork(), you may disable this warning
by setting the mpi_warn_on_fork MCA parameter to 0.
--------------------------------------------------------------------------


0

In [3]:
#Make PDB for each chain
if not os.path.isdir('pdb'): os.system('mkdir pdb')
if not os.path.isfile('{}_a.pdb'.format(pdbid)):
    for chain in chains:
        if pdbid == '/home/trowray/acrb/pdb/1iwg': 
            os.system('convpdb.pl -out charmm22 -nsel {}: -segnames {}.pdb > {}_{}.pdb'\
                                    .format(chain,pdbid,pdbid,chain.lower())) # Note .lower() to fix filenames
            os.system('sed -i "s/HSD {} 672/HSP {} 672/g" {}_{}.pdb'.format(chain.upper(), chain.upper(), pdbid, chain.lower()))
        else: 
            os.system('convpdb.pl -out charmm22 -nsel {} -segnames {}.pdb > {}_{}.pdb'\
                        .format('protein',pdbid,pdbid,chain.lower()))  # Note .lower() to fix filenames
        segids.append(get_assigned_segid(file='{}_{}.pdb'.format(pdbid,chain.lower())))

In [4]:
read.rtf('/home/trowray/acrb/toppar/top_all36_prot.rtf')
read.prm('/home/trowray/acrb/toppar/par_all36m_prot.prm', flex=True)
pycharmm.lingo.charmm_script('stream /home/trowray/acrb/toppar/toppar_water_ions.str')

  
 CHARMM>     read rtf card -
 CHARMM>     name /home/trowray/acrb/toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::/home/trowray/acrb/toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name /home/trowray/acrb/toppar/par_all36m_prot.prm -
 CHARMM>     flex
 VOPEN> Attempting to open::/home/trowray/acrb/toppar/par_all36m_prot.prm::

          PARAMETER FILE BEING READ FROM UNIT 91
 TITLE> *>>>> CHARMM36 ALL-HYDROGEN PARAMETER FILE FOR PROTEINS <<<<<<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TER

1

In [5]:
#Chain coordinates
# Loop over the segments identified in the pdb files
for ichain,segid in enumerate(segids):
    # read in the sequence of the protein to be generated
    # only useful for the same residue
    # equivalent to the CHARMM scripting command:
    # read sequ pdb name pdb/{}_{}.pdb
    read.sequence_pdb('{}_{}.pdb'.format(pdbid,chains[ichain].lower())) # Note .lower() to fix filenames

    # equivalent to the CHARMM scripting command: generate ADP first ACE last CT3 setup
    gen.new_segment(seg_name=segid, first_patch='ACE', last_patch='CT3', setup_ic=True)
    # equivalent to read coor pdb name pdb/{}_{}.pdb resid
    read.pdb('{}_{}.pdb'.format(pdbid,chains[ichain].lower()),resid=True) # Note .lower() to fix filenames
    # equivalent to the CHARMM scripting command: ic param
    ic.prm_fill(replace_all=False)
    # equivalent to the CHARMM scripting command: ic build
    ic.build()
# add disulfide bonds in present
# note, we do this after we generate the sequences since there may be inter-chain disulfides
if len(disu.keys())>0:
    for ssbond in disu.keys():
        pycharmm.lingo.charmm_script('patch disu pro{} {} pro{} {}'\
                                     .format(disu[ssbond]['chains'][0],disu[ssbond]['resnums'][0],
                                             disu[ssbond]['chains'][1],disu[ssbond]['resnums'][1]))
# The coor orie command is useful to expose since it allows one to
# orient the system in preparation for other calculations
# equivalent to the CHARMM scripting command: coor orient
coor.orient(by_rms=False,by_mass=False,by_noro=False)
# equivalent to the CHARMM scripting command: print coor
coor.show()
# If pdb directory doesn't already exist make it here.
if not os.path.isdir('pdb'): os.system('mkdir pdb')
# equivalent to the CHARMM scripting command: write coor pdb name pdb/pdbid_initial.pdb
write.coor_pdb('{}_initial.pdb'.format(pdbid))

  
 CHARMM>     read sequence pdb -
 CHARMM>     name /home/trowray/acrb/pdb/1iwg_a.pdb
 VOPEN> Attempting to open::/home/trowray/acrb/pdb/1iwg_a.pdb::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *
SEQRDR> Residue insertion at RESID "1000"
SEQRDR> Residue insertion at RESID "1001"
SEQRDR> Residue insertion at RESID "1002"
SEQRDR> Residue insertion at RESID "1003"
SEQRDR> Residue insertion at RESID "1004"
SEQRDR> Residue insertion at RESID "1005"
SEQRDR> Residue insertion at RESID "1006"

          RESIDUE SEQUENCE --  1006 RESIDUES
          ASP ARG PRO ILE PHE ALA TRP VAL ILE ALA ILE ILE ILE MET LEU ALA GLY GLY LEU ALA 
          ILE LEU LYS LEU PRO VAL ALA GLN TYR PRO THR ILE ALA PRO PRO ALA VAL THR ILE SER 
          ALA SER TYR PRO GLY ALA ASP ALA LYS THR VAL GLN ASP THR VAL THR GLN VAL ILE GLU 
          GLN ASN MET ASN GLY ILE ASP ASN LEU MET TYR MET SER SER ASN SER ASP SER THR GLY 
          THR VAL GLN ILE THR LEU THR PHE GLU SER GLY THR ASP ALA ASP ILE ALA

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
#Minimization
# Specify nonbonded python object called my_nbonds - this just sets it up
# equivalant CHARMM scripting command: nbonds cutnb 18 ctonnb 13 ctofnb 17 cdie eps 1 atom vatom fswitch vfswitch
my_nbonds = pycharmm.NonBondedScript(
    cutnb=18.0, ctonnb=13.0, ctofnb=17.0,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True)


# Implement these non-bonded parameters by "running" them.
my_nbonds.run()
# equivalent to: cons harm force 20 select type ca end
cons_harm.setup_absolute(selection=pycharmm.SelectAtoms(atom_type='CA'),force_constant=20)
# equivalent CHARMM scripting command: minimize abnr nstep 1000 tole 1e-3 tolgr 1e-3
minimize.run_sd(nstep=300, tolenr=1e-3, tolgrd=1e-3)
# equivalent CHARMM scripting command: energy
energy.show()
# equivalent to the CHARMM scripting command: write coor pdb name pdb/initial.pdb
write.coor_pdb('{}_minimized.pdb'.format(pdbid))
write.psf_card('{}.psf'.format(pdbid))

  
 CHARMM>     nbonds cutnb 18.0 -
 CHARMM>     ctonnb 13.0 -
 CHARMM>     ctofnb 17.0 -
 CHARMM>     eps 1.0 -
 CHARMM>     cdie -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     fswitch -
 CHARMM>     vfswitch

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSWItch  VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 18.000 CTEXNB =999.000 CTONNB = 13.000 CTOFNB = 17.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 <MAKINB> with mode   5 found 131694 exclusions and 122019 interactions(1-4)
 <MAKGRP> found  41685 group exclusions.
 Generating nonbond list with Exclusion mode = 5
 == PRIMARY == SPACE FOR 28318190 ATOM PAIRS AND        0 GROUP PAIRS
 == PRIMARY == SPACE FOR 42477304 ATOM PAIRS AND        0 GROUP PAIRS

 General atom nonbond list generation foun

In [ ]:
#Ions but not working - Line 4
import numpy as np
# find the overall charge so we can add neutralizing ions
q = 6
Ntot = round((np.sum(q)))
if Ntot > 0: ion_type = 'CLA'
if Ntot < 0: ion_type = 'SOD'
ions = '-ions {}:{}'.format(ion_type,np.abs(Ntot))
if np.abs(Ntot) < 1e-2: ions = ''

In [ ]:
#Solvation

# CHARMM scripting command: system "convpdb.pl -solvate {ions} -cutoff 10 -cubic -out charmm22 pdb/adp.pdb
# | convpdb.pl -segnames -nsel TIP3 > pdb/{}_wt00.pdb"
solvate_command = 'convpdb.pl -solvate -cutoff 5 {} -cubic -out charmm22 {}_minimized.pdb > /home/trowray/acrb/pdb/w.pdb;'\
    .format(ions,pdbid)
solvate_command +='convpdb.pl -segnames -nsel TIP3 /home/trowray/acrb/pdb/w.pdb > {}_wt00.pdb;'.format(pdbid)
solvate_command +='convpdb.pl -segnames -nsel ion /home/trowray/acrb/pdb/w.pdb > ions.pdb'
# run the command as a system subprocess
os.system(solvate_command)
# replace HETATM by ATOM in ions
fpdb = open('ions.pdb','r')
opdb = open('{}_ions.pdb'.format(pdbid),'w')
for l in fpdb:
    print(l.strip().replace('HETATM','ATOM  '),file=opdb)
fpdb.close()
opdb.close()
# clean-up non-specific files
os.system('rm /home/trowray/acrb/pdb/ions.pdb')

In [ ]:
#Fix Multiple Water Segment ID's
os.system('cat /home/trowray/acrb/pdb/1iwg_wt00.pdb | grep WT01 > /home/trowray/acrb/pdb/1iwg_wt01.pdb')
os.system('cat /home/trowray/acrb/pdb/1iwg_wt00.pdb | grep WT02 > /home/trowray/acrb/pdb/1iwg_wt02.pdb')
os.system('cat /home/trowray/acrb/pdb/1iwg_wt00.pdb | grep WT03 > /home/trowray/acrb/pdb/1iwg_wt03.pdb')
os.system('cat /home/trowray/acrb/pdb/1iwg_wt00.pdb | grep WT04 > /home/trowray/acrb/pdb/1iwg_wt04.pdb')
os.system('cat /home/trowray/acrb/pdb/1iwg_wt00.pdb | grep WT05 > /home/trowray/acrb/pdb/1iwg_wt05.pdb')
os.system('cat /home/trowray/acrb/pdb/1iwg_wt00.pdb | grep WT06 > /home/trowray/acrb/pdb/1iwg_wt06.pdb')
os.system('cat /home/trowray/acrb/pdb/1iwg_wt00.pdb | grep WT07 > /home/trowray/acrb/pdb/1iwg_wt07.pdb')
os.system('cat /home/trowray/acrb/pdb/1iwg_wt00.pdb | grep WT08 > /home/trowray/acrb/pdb/1iwg_wt08.pdb')
os.system('mv /home/trowray/acrb/pdb/1iwg_wt00.pdb /home/trowray/acrb/pdb/1iwgwt00.pdb')
os.system('cat /home/trowray/acrb/pdb/1iwgwt00.pdb | grep WT00 > /home/trowray/acrb/pdb/1iwg_wt00.pdb')
os.system('rm /home/trowray/acrb/pdb/1iwgwt00.pdb')

os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt00.pdb')
os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt01.pdb')
os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt02.pdb')
os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt03.pdb')
os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt04.pdb')
os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt05.pdb')
os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt06.pdb')
os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt07.pdb')
os.system('echo "TER\nEND" >> /home/trowray/acrb/pdb/1iwg_wt08.pdb')

In [ ]:
#Water
y = 0
for i in range(0,9):
    water_segment = get_assigned_segid(file='/home/trowray/acrb/pdb/1iwg_wt0' + str(y) + '.pdb')
    old_wrnlev = settings.set_warn_level(0)
    read.sequence_pdb('/home/trowray/acrb/pdb/1iwg_wt0' + str(y) + '.pdb')
    # Now reset back to default wrnlev
    settings.set_warn_level(old_wrnlev)
    # Another example of the generate command
    # generate wt00 noangle nodihedral
    gen.new_segment(water_segment, angle=False, dihedral=False)
    # read coor pdb name pdb/pdbid_wt00.pdb resid
    read.pdb('/home/trowray/acrb/pdb/1iwg_wt0' + str(y) + '.pdb'.format(pdbid), resid=True)
    y += 1

# Here is an alternative means of reading a sequence
# read sequ pdb name pdb/{}_ions.pdb
# get ion sequence name
ion_segment = get_assigned_segid(file='{}_ions.pdb'.format(pdbid))
read.sequence_pdb('{}_ions.pdb'.format(pdbid))

# Another example of the generate command
# generate wt00 noangle nodihedral
gen.new_segment(ion_segment, angle=False, dihedral=False)

# read coor pdb name pdb/adp.pdb resid
read.pdb('{}_ions.pdb'.format(pdbid), resid=True)
# get the coor statistics to construct boxlengths
# coor stat
stats = coor.stat()

0

In [ ]:
# Minimization of Solvated System

# boxsize
xsize = stats['xmax'] - stats['xmin']
ysize = stats['ymax'] - stats['ymin']
zsize = stats['zmax'] - stats['zmin']
boxsize = max(xsize, ysize, zsize)

# half box size
boxhalf = boxsize / 2.0

# CHARMM scripting: crystal define cubic @boxsize @boxsize @boxsize 90 90 90
crystal.define_cubic(boxsize)
# CHARMM scripting: crystal build cutoff @boxhalf noper 0
crystal.build(boxhalf)

# Turn on image centering - bysegment for protein, by residue for solvent and ions
# CHARMM scripting: image byseg xcen 0 ycen 0 zcen 0 select segid SEGID end
for segid in segids:
    image.setup_segment(0.0, 0.0, 0.0, segid)
# CHARMM scripting: image byres xcen 0 ycen 0 zcen 0 select resname tip3 end
image.setup_residue(0.0, 0.0, 0.0, 'TIP3')
# CHARMM scripting: image byres xcen 0 ycen 0 zcen 0 select resname ion_type end
image.setup_residue(0.0, 0.0, 0.0, ion_type)

# Now specify nonbonded cutoffs for solvated box
cutnb = min(boxhalf,12)
cutim = cutnb
ctofnb = cutnb - 1.0
ctonnb = cutnb - 3.0

# Another nbonds example
# CHARMM scripting: nbonds cutnb @cutnb cutim @cutim ctofnb @ctofnb ctonnb @ctonnb -
#        inbfrq -1 imgfrq -1
pycharmm.NonBondedScript(
    cutnb=cutnb, cutim=cutim, ctonnb=ctonnb, ctofnb=ctofnb,
    eps=1.0,
    cdie=True,
    atom=True, vatom=True,
    fswitch=True, vfswitch=True,
    inbfrq=-1, imgfrq=-1).run()

# Fix the peptide and minimize the solvent to "fit"
# CHARMM scripting: cons fix select segid adp end
cons_fix.setup(pycharmm.SelectAtoms(seg_id=segids))

# Minimize the solvent positions with periodic boundary conditions using steepest descents
# CHARMM scripting: mini sd nstep 200 tole 1e-3 tolgrd 1e-3
minimize.run_sd(nstep=200, tolenr=1e-3, tolgrd=1e-3)

# Turn off fixed atoms
# CHARMM scripting: cons fix select none end
cons_fix.turn_off()

# Write the psf and coordinates for the solvated peptide
# write psf card name pdb/adp+wat.psf
write.psf_card('{}+wat.psf'.format(pdbid))
# write coor pdb name pdb/adp+wat_min.pdb
# write.coor_pdb('{}+wat_min.pdb'.format(pdbid))
write.coor_card('{}+wat_min.crd'.format(pdbid))

: 